In [ ]:
!pip install SimpleITK
!pip install skan #older version
!pip install tifffile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import SimpleITK as sitk
from skimage.morphology import skeletonize, thin, medial_axis
from skan import skeleton_to_csgraph, Skeleton
from skan import summarize
from skan import draw
import tifffile as tiff

import matplotlib.pyplot as plt
import numpy as np
import cv2
import plotly.express as px
import pandas as pd

In [5]:
def load_img_from_tiff(path2img):
    """
    Parameters
    ----------
    path2img: str 
        path to image Tiff file
    
    Returns
    -------
    img_array: np.array
        image data in numpy format
    """
    img = sitk.ReadImage(path2img)
    img_array = sitk.GetArrayFromImage(img)
    return(img_array)

Skeletonize single frame

In [65]:
single_thresh = load_img_from_tiff("/content/drive/MyDrive/mydata/sea_urchin_data/Test n.1")
# single_thresh = cv2.resize(single_thresh, dsize=(256,256), interpolation=cv2.INTER_CUBIC)

single_thresh = single_thresh.astype('float64')
single_thresh = (single_thresh - np.min(single_thresh))/np.ptp(single_thresh)
skimage_skeleton, dist = medial_axis(single_thresh, return_distance=True)
skan_skeleton = Skeleton(dist*skimage_skeleton)
df=summarize(skan_skeleton)
# df.drop(df[df["branch-type"] < 2].index, inplace=True)

In [66]:
# plt.imshow(single_thresh, cmap='gray')
# plt.title("Thresholded Image")

In [67]:
#rename columns:
df = df.rename(columns={'image-coord-src-0': 'src-y', 
                        'image-coord-src-1': 'src-x',
                        'image-coord-dst-0': 'dst-y',
                        'image-coord-dst-1': 'dst-x',
                        'mean-pixel-value': 'thickness'})

df.drop([#'mean-pixel-value',
         'stdev-pixel-value',
         'coord-src-0',
         'coord-src-1',
         'coord-dst-0',
         'coord-dst-1'], axis=1, inplace=True)

In [68]:
# plt.rcParams["figure.figsize"] = (15,15)
# plt.imshow(single_thresh+skimage_skeleton, cmap='gray')
# plt.title("Medial Axis Skeleton")

In [ ]:
#overlay nodes 
plt.rcParams["figure.figsize"] = (15,15)
res_st = cv2.resize(single_thresh, dsize=(256,256), interpolation=cv2.INTER_CUBIC)
xs=np.array(df['src-x'])
ys=np.array(df['src-y'])
node_image = np.zeros([305,305], dtype=np.uint8)
# print(node_image.shape)
node_image[ys.astype(np.uint16), xs.astype(np.uint16)] = 4.
plt.imshow(node_image+skimage_skeleton+single_thresh, cmap='inferno')
plt.title("Branch Nodes")
# plt.scatter(ys,xs)

In [70]:
# plt.imshow(dist*skimage_skeleton+single_thresh, cmap='inferno')
# plt.title("Distance from nearest void (skeleton)")
# plt.colorbar()

In [71]:
def get_slope(x1, y1, x2, y2):
    if x1==x2:
        return ""
    m = (y2-y1)/(x2-x1)
    return m

df["slope"] = 0
df["tortuosity"] = 0

for index, row in df.iterrows():
    df.loc[index, 'slope']=get_slope(row["src-x"], row["src-y"], row["dst-x"], row["dst-y"])
    df.loc[index, 'tortuosity']=row["branch-distance"]/row["euclidean-distance"]

df.to_csv("single_summary.csv")

In [72]:
nodes = pd.DataFrame(df["node-id-src"])
nodes = nodes.rename(columns={'node-id-src': 'node_id'})
nodes["node_coordinate_x"]=xs
nodes["node_coordinate_y"]=ys
#nodes=nodes.drop_duplicates(subset=['node_id', 'node_coordinate_x', 'node_coordinate_y'], keep=False)
nodes = nodes.reset_index()
nodes = nodes.drop(['index'], axis=1)

In [ ]:
df2 = {'node_id': list(df['node-id-dst'])[-1],
       'node_coordinate_x': list(df['dst-x'])[-1],
       'node_coordinate_y': list(df['dst-y'])[-1],
       }
nodes = nodes.append(df2, ignore_index = True)
nodes

In [ ]:
# generate_node_image
xs_=np.array(nodes['node_coordinate_x'])
ys_=np.array(nodes['node_coordinate_y'])
node_image = np.zeros([305,305], dtype=np.uint8)
# print(node_image.shape)
node_image[ys_.astype(np.uint16), xs_.astype(np.uint16)] = 4.
plt.rcParams["figure.figsize"] = (10,10)
plt.imshow(node_image + skimage_skeleton, cmap='gray')

In [75]:
nodes.to_csv("nodes.csv", index=False)

In [ ]:
branches = pd.DataFrame(df['thickness'])
branches['source_node_id']=df['node-id-src']
branches['destination_node_id']=df['node-id-dst']
branches

In [78]:
branches.to_csv("branches.csv", index=False)

In [ ]:
#compute porosity
def compute_porosity_2d(img):
    number_of_white_pix = np.sum(img == 1)  
    number_of_black_pix = np.sum(img == 0)
    porosity=number_of_black_pix/(number_of_white_pix+number_of_black_pix)
    return(porosity)

compute_porosity_2d(single_thresh)

Skeletonize multiple Z slices



In [ ]:
path = "/content/drive/MyDrive/mydata/sea_urchin_data/3D/Galleria Piastra Modello 1 100x100 Echi1-10x.tif"
data = load_img_from_tiff(path)
for i in range(len(data)):
    data[i] = data[i]*(255.0/data[i].max())

In [ ]:
blurred_data = np.zeros_like(data)
for i in range(len(blurred_data)):
    blurred_data[i] = cv2.GaussianBlur(data[i], (3,3),3)

thresholded_data = np.zeros_like(data)
for i in range(len(thresholded_data)):
    bin, thresholded_data[i] = cv2.threshold(blurred_data[i],175, 255, cv2.THRESH_BINARY)

medial_axis_skel = np.zeros_like(data)
for i in range(len(thresholded_data)):
    # skel, distance = medial_axis(thresholded_data[i], return_distance=True)
    # dist_on_skel = distance * skel
    # medial_axis_skel[i]=skel
    skel = skeletonize(thresholded_data[i]//255)
    medial_axis_skel[i]=skel

In [ ]:
skan_skel = Skeleton(medial_axis_skel)
df=summarize(skan_skel)
df.to_csv("z_slices_summary.csv")
#drop brances with length < 2
# df.drop(df[df["branch-distance"] < 2].index, inplace=True)
df